# Déploiement d'un modèle Azure ML service avec Azure Container Instance (ACI)

<img src='https://cdn.thenewstack.io/media/2018/10/2e4f0988-az-ml-0.png'>

## 1. Connexion workspace

In [1]:
import sys
sys.version

'3.6.2 |Anaconda, Inc.| (default, Sep 30 2017, 18:42:57) \n[GCC 7.2.0]'

In [2]:
import time
datedujour = time.strftime("%Y-%m-%d")
print(datedujour)

2019-10-24


In [3]:
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.0.65


In [4]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

Ready to work with azuremlservice


## 2. Déploiement modèle

In [5]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

WARNING - Cannot find Saved Dataset with ID c3eba373-f459-49b3-a175-ec2b553ff8ce due to the following exception.
Operation returned an invalid status code "Exception of type 'Common.WebApi.Exceptions.ResourceNotFoundException' was thrown."
WARNING - Cannot find Saved Dataset with ID c3eba373-f459-49b3-a175-ec2b553ff8ce due to the following exception.
Operation returned an invalid status code "Exception of type 'Common.WebApi.Exceptions.ResourceNotFoundException' was thrown."


diabetes_model version: 14
	 Training context : custom local environment
	 AUC : 0.8568646196561496
	 Accuracy : 0.7893333333333333


sklearn_regression_model.pkl version: 50
	 area : diabetes
	 type : regression
	 run_id : devops-ai-demo_1571878963_ad0bd6a4


sklearn_regression_model.pkl version: 49
	 area : diabetes
	 type : regression
	 run_id : devops-ai-demo_1571792556_c5e847b2


best_model_exemple version: 6


best_model_exemple version: 5


usedcarsmodel version: 17


usedcarsmodel version: 16


sklearn_regression_model.pkl version: 48
	 area : diabetes
	 type : regression
	 run_id : devops-ai-demo_1571735714_39f36de7


best_model_exemple version: 4


AutoML100842e50best version: 1


AutoMLe8eb9dec89 version: 1


sklearn_regression_model.pkl version: 47
	 area : diabetes
	 type : regression
	 run_id : devops-ai-demo_1571360562_f2f5309e


best_model_exemple version: 3


sklearn_regression_model.pkl version: 46
	 area : diabetes
	 type : regression
	 run_id : devops-ai-demo_157127

In [6]:
model = ws.models['diabetes_model']
print(model.name, 'version', model.version)

WARNING - Cannot find Saved Dataset with ID c3eba373-f459-49b3-a175-ec2b553ff8ce due to the following exception.
Operation returned an invalid status code "Exception of type 'Common.WebApi.Exceptions.ResourceNotFoundException' was thrown."
WARNING - Cannot find Saved Dataset with ID c3eba373-f459-49b3-a175-ec2b553ff8ce due to the following exception.
Operation returned an invalid status code "Exception of type 'Common.WebApi.Exceptions.ResourceNotFoundException' was thrown."


diabetes_model version 14


In [7]:
import os

folder_name = 'diabetes_service'

# Create a folder for the web service files
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

print(folder_name, 'folder created.')

diabetes_service folder created.


In [8]:
%%writefile $folder_name/score_diabetes.py
import json
import numpy as np
import os
import joblib
from azureml.core.model import Model
import azureml.train.automl # Required for AutoML models

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data - the features of patients to be classified.
    data = json.loads(raw_data)['data']
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Writing diabetes_service/score_diabetes.py


In [9]:
from azureml.core.conda_dependencies import CondaDependencies 

# Add the dependencies for our model (AzureML defaults is already included)
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
myenv.add_pip_package("azureml-sdk[automl]") # Required for AutoML models

# Save the environment config as a .yml file
env_file = folder_name + "/diabetes_env.yml"
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

# Print the .yml file
with open(env_file,"r") as f:
    print(f.read())

Saved dependency info in diabetes_service/diabetes_env.yml
# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

  - azureml-sdk[automl]
- scikit-learn
channels:
- conda-forge



In [10]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   source_directory = folder_name,
                                   entry_script="score_diabetes.py",
                                   conda_file="diabetes_env.yml")

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "diabetesservice"

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Running...............................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy


## 3. Call du webservice

In [22]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print("Model prediction=",predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
Model prediction= not-diabetic


In [21]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), "==> Model prediction=", predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] ==> Model prediction= not-diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] ==> Model prediction= not-diabetic


In [13]:
endpoint = service.scoring_uri
print(endpoint)

http://a040a79a-53ca-4093-9b0d-2a085e908cc7.westeurope.azurecontainer.io/score


In [20]:
import requests
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [1,148,58,11,100,39.19207553,0.160829008,75]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), "==> Model prediction =", predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] ==> Model prediction = not-diabetic
Patient [1, 148, 58, 11, 100, 39.19207553, 0.160829008, 75] ==> Model prediction = diabetic


> Fin